In [2]:
from housing_pricer.scraping.sdk.data_manager import DataManager
from housing_pricer.data_processing.data_processing_utils import format_json_to_dataframe
import pandas as pd

pd.set_option('display.max_columns', None)
DATA_STORAGE = "../scraping/data_storage"


listings = format_json_to_dataframe(DataManager(DATA_STORAGE).load_data())
listings

Processing scraped JSON content to dataframe...: 25995it [00:01, 23170.58it/s]
